In [ ]:
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil

import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from ROOT import larocv
from ROOT import std

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
proc = larcv.ProcessDriver('ProcessDriver')

CFG="../reco_shower.cfg"
print "Loading config... ",CFG
proc.configure(CFG)

flist=ROOT.std.vector('std::string')()
flist.push_back("/Users/vgenty/Desktop/nue_8000.root")
proc.override_input_file(flist)

filter_id = proc.process_id("NuFilter")
mcinfo_id = proc.process_id("LArbysImageMC")
reco_id   = proc.process_id("LArbysImage")
ana_id    = proc.process_id("LArbysImageAna")

filter_proc   = proc.process_ptr(filter_id)
mcinfo_proc   = proc.process_ptr(mcinfo_id)
mcinfo_proc.SetFilter(filter_proc)

larbysimg     = proc.process_ptr(reco_id)
larbysimg_ana = proc.process_ptr(ana_id)
larbysimg_ana.SetManager(larbysimg.Manager())

proc.override_ana_file("/tmp/test.root")

proc.initialize()

for event in xrange(10):
    proc.batch_process(event,1)
    if (filter_proc.selected()==False): continue
    print event
    break


In [ ]:
mgr=larbysimg.Manager()
img_v = []
track_img_v=[]
shower_img_v=[]
pygeo = geo2d.PyDraw()
for mat in mgr.InputImages(0):
    img_v.append(pygeo.image(mat))
for mat in mgr.InputImages(1):
    track_img_v.append(pygeo.image(mat))
for mat in mgr.InputImages(2):
    shower_img_v.append(pygeo.image(mat))

In [ ]:
for plane in xrange(len(track_img_v)):                                                                                                    
        shower_img=np.where(shower_img_v[plane]>10.0,85.0,0.0).astype(np.uint8)                                                                 
        track_img=np.where(track_img_v[plane]>10.0,160.0,0.0).astype(np.uint8)                                                                  
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')                                                                                      
        plt.imshow(shower_img+track_img,cmap='jet',interpolation='none',vmin=0.,vmax=255.)                                                      
        plt.xlabel('Time [6 ticks]',fontsize=20)                                                                                                
        plt.ylabel('Wire',fontsize=20)                                                                                                          
        plt.tick_params(labelsize=20)                                                                                                           
        ax.set_aspect(0.8)                                                                                                                      
        plt.tight_layout()        
        plt.show()